In [15]:
import warnings

import pandas as pd
import plotly.express as px

warnings.filterwarnings('ignore')

In [17]:

change_dtype = {5: str, 32: "category", 34: "category", 55: str, 62: str, 63: str, 64: str, 77: "category",
                80: "category", 91: "category", 93: "category", 95: "category", 97: str, 115: str, 116: "category",
                122: "category"}

df: pd.DataFrame = pd.read_csv("../data/raw/globalterrorismdb_0221dist.csv", index_col=0, dtype=change_dtype)

In [40]:
df.describe(include="all").T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
eventid,139969.0,NaN,NaN,NaN,200584561134.368317,1253323098.138676,197000000001.0,199602080007.0,201203130034.0,201509040003.0,201912310033.0
iyear,139969.0,NaN,NaN,NaN,2005.779465,12.53347,1970.0,1996.0,2012.0,2015.0,2019.0
imonth,139969.0,NaN,NaN,NaN,6.459216,3.388205,0.0,4.0,6.0,9.0,12.0
iday,139969.0,NaN,NaN,NaN,15.551829,8.798552,0.0,8.0,15.0,23.0,31.0
extended,139969.0,NaN,NaN,NaN,0.036158,0.186684,0.0,0.0,0.0,0.0,1.0
country,139969.0,NaN,NaN,NaN,126.06495,104.49968,4.0,78.0,95.0,160.0,1004.0
country_txt,139969,200,Iraq,21993,NaN,NaN,NaN,NaN,NaN,NaN,NaN
region,139969.0,NaN,NaN,NaN,7.286406,2.803477,1.0,6.0,6.0,10.0,12.0
region_txt,139969,12,Middle East & North Africa,40337,NaN,NaN,NaN,NaN,NaN,NaN,NaN
provstate,139969,2311,Baghdad,7226,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# filtrowanie kolumn i wyrzucenie tych w których większość wierszy jest pusta
df = df[(df.crit1 == 1) & (df.crit2 == 1) & (df.crit3 == 1) & (df.doubtterr == 0)]
df.dropna(axis=1, thresh=int(0.5 * df.shape[0]), inplace=True)

In [19]:
# Usunięcie i zamiana niespójnie wprowadzonych danych
df = df[df.vicinity != -9]
df = df[df.claimed != -9]
df.nperps = df.nperps.apply(lambda x: -99 if x == -9 else x)
df.nperpcap = df.nperpcap.apply(lambda x: -99 if x == -9 else x)

# usuniecie wartości pustych z kolumn docelowych
df.dropna(subset=["nkill", "nwound"], inplace=True)
# wypełnienie wartosci pustych zerami
df["nkillter"].fillna(0, inplace=True)
df["nwoundte"].fillna(0, inplace=True)
# usunięcie z ogólnej liczby zgonów/rannych terrorystów
df["nkill"] = df["nkill"] - df["nkillter"]
df["nwound"] = df["nwound"] - df["nwoundte"]
# Utworzenie kolumn z sumaryczną liczba zgonów i ofiar oraz klasy zerojedynkowej.
df["ncasualties"] = df["nkill"] + df["nwound"]
df["cas_class"] = df["ncasualties"].apply(lambda x: 0 if x == 0 else 1)

In [39]:
df.to_csv("../data/interm/globalterrordb.csv")